# Azure Function to build Storage Accounts

We have a table that contains the details of the required storage accounts - let's have Azure SQL build them out.

Requirements:
- The Azure SQL DB table should be the 'system of record' - so the single source of truth
- Adding rows should create a storage account
- Deleting rows should remove the storage account
- Modifying rows?

Questions remain in this example:
- How does the data get here?
    - Maybe another API to insert data - from change management system
    - A front end web application that lets users request storage accounts
    - ??



## PreReqs
- Create the Azure Function to do the work
- Configure Authentication
    - There are options - best option is Managed Identity
        - Enable Managed Identity for Azure SQL Database
            - This creates an app id to use for authentication
        - Enabled Azure AD authentication for Azure Function

## Setup

1. Create a database master key if one doesn't exist - with a proper strong password

In [ ]:
-- create a database master key
IF NOT EXISTS(SELECT * FROM sys.symmetric_keys WHERE name = '##MS_DatabaseMasterKey##')
BEGIN
    create master key encryption by password = '*********'
END

2. Store the app id for the Azure SQL DB as a `DATABASE SCOPED CREDENTIAL`

In [ ]:
-- create a database scoped credential for managed identity (or Request Headers or Query String)
CREATE DATABASE SCOPED CREDENTIAL [https://psconfeu2023.azurewebsites.net/api/NewStorageAcct]
WITH IDENTITY = 'Managed Identity', SECRET = '{"resourceid":"GUID-GUID-GUID-GUID-GUID"}';

3. Call `sp_invoke_external_rest_endpoint` to create a storage account

In [ ]:
set @url = 'https://psconfeu2023.azurewebsites.net/api/NewStorageAcct?name=' + LOWER(@Name)

declare @ret as int, @response as nvarchar(max);

exec @ret = sp_invoke_external_rest_endpoint 
	@method = 'GET',
	@url = @url,
    @timeout = 230, -- max value which is 3.8 mins
    @credential = [https://psconfeu2023.azurewebsites.net/api/NewStorageAcct],
 	@response = @response output;
	
select @ret as ReturnCode, @response as Response;

exec @ret = sp_invoke_external_rest_endpoint 
	@method = 'GET',
	@url = @url,
 	@response = @response output;

GO